In [362]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [363]:
# Define mean vector and covariance matrix
mean = np.array([0, 0])
cov = np.array([[1, 0], [0, 2]])
v0 = 5
del_0 = np.array([[4,0],[0,5]])

# Generate samples
N = 10000
samples = np.random.multivariate_normal(mean, cov, N)

In [364]:
samples.shape

(10000, 2)

## Helper Functions

In [365]:
def get_sym_matrix(cov_matrix,d):
    """
    Get the symmetric matrix from the covariance matrix.
    """
    cov_sym_matrix = np.zeros((d,d))
    cov_sym_matrix[0][0],cov_sym_matrix[0][1],cov_sym_matrix[1][0],cov_sym_matrix[1][1] = cov_matrix[0],cov_matrix[1],cov_matrix[1],cov_matrix[2] 
    return cov_sym_matrix

def ispsd(matrix):
    """
    Check if a matrix is positive semi-definite.
    
    Parameters:
    matrix (ndarray): Matrix to check.
    
    Returns:
    bool: True if the matrix is positive semi-definite, False otherwise.
    """
    return np.all(np.linalg.eigvals(matrix) > 0)

def Wishart(samples, cov_matrix,v_n,del_n):
    """
    Calculate the log-likelihood of the Wishart distribution.
    samples: array-like, shape (n_samples, n_features)
    cov_matrix: array-like, shape (n_features, n_features)
    v_n: float, degrees of freedom
    del_n: array-like, shape (n_features, n_features)
    """
    _,d = samples.shape
    cov_sym_matrix = get_sym_matrix(cov_matrix,d)
    if not ispsd(cov_sym_matrix):
        return 1000000

    determinant = np.linalg.det(cov_sym_matrix)    
    return np.log(determinant)*((v_n + d + 1) / 2) +0.5* np.trace(del_n @ np.linalg.inv(cov_sym_matrix)) 

## Estimators

In [366]:
def MLEestimate(samples,mu):
    """
    Maximum Likelihood Estimation of the covariance matrix
    :param samples: 2D array of samples
    :param mu: mean vector
    :return: covariance matrix
    """
    n,d = samples.shape[0],samples.shape[1]
    return (samples-mu.reshape(1,d)).T @ (samples-mu.reshape(1,d)) / n

def BayesEstimateUsingtPrior(samples,cov_guess,v0,del_0):
    """
    Bayesian estimation of the covariance matrix using a Wishart prior
    :param samples: 2D array of samples
    :return: covariance matrix
    """
    n = samples.shape[0]
    initial_guess = (cov_guess).reshape(-1)
    initial_guess = initial_guess[[0,1,3]]
    v_n = v0 + n 
    del_n = del_0 + samples.T @ samples
    ret = minimize(lambda x: Wishart(samples, x, v_n, del_n),initial_guess,tol=1e-6)
    return ret

In [367]:
cov = np.array([[7, 4], [4,8]])
# ans = BayesEstimateUsingWishartPrior(samples, mean,cov)

In [368]:
ans = BayesEstimateUsingtPrior(samples, cov,v0,del_0)
ans = get_sym_matrix(ans.x,2)
ans


array([[9.94403861e-01, 5.78466455e-04],
       [5.78466455e-04, 1.98833591e+00]])

In [369]:
ans = BayesEstimateUsingtPrior(samples, cov,1,np.zeros((2,2)))
ans = get_sym_matrix(ans.x,2)
ans


array([[9.94401651e-01, 5.78686081e-04],
       [5.78686081e-04, 1.98863128e+00]])

In [370]:
ans = BayesEstimateUsingtPrior(samples, cov,0,np.zeros((2,2)))
ans = get_sym_matrix(ans.x,2)
ans

array([[9.94501062e-01, 5.78734921e-04],
       [5.78734921e-04, 1.98883007e+00]])